In [13]:
# !git clone https://github.com/openai/CLIP.git


# !git clone https://github.com/CompVis/taming-transformers


Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 8.93 MiB | 506.00 KiB/s, done.
Resolving deltas: 100% (133/133), done.
Updating files: 100% (22/22), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1342, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
error: 4253 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [1]:
# Some More installs

!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
!pip uninstall torchtext --yes
!pip install einops



#### Imports


In [10]:
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as F

import torch
!pip3 install imageio
import os, tqdm, imageio, pdb, math

import numpy as np

# viz imports
!pip3 install matplotlib
import matplotlib.pyplot as plt
import PIL


from omegaconf import OmegaConf
import yaml

In [6]:
# supress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# 1. We need intermediate results to be shown for each iteration
# 2. normalise and bring (-1, 1) to (0, 0.5)
# 3. optimisation and reguralisation params


def show_tensor(tensor):
    img = tensor.clone()
    img = img.mul(255) # (0, 1) before for faster and stable training
    img = img.cpu().numpy().transpose(1, 2, 0)

    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

def norm(x):
    return (x.clip(-1, 1) + 1)/2 # 1st clip within -1 to 1 then (0, 2) by adding 1, then (0, 1)

# optimisation params
wd = 0.1 # recall this is generalisation param, where penalise larger weights
# Loss = Original_Loss + lambda * W^2
# Larger weights basically memorise the training data. How?
# Larger weights make model highly sensitive to smaller changes in training data
# resulting in capturing the noise as well on top of capturing the patterns
# Smaller weights means => Less variance in model's predictions => more stable across datasets
# => Less sensitive to noise
# Penalising higher weights makes the model find a sweet-spot between good accuracy 
# and more generalisation
lr = 0.5
batch_size = 1
total_iterations = 100 # good starting point
noise_factor = 0.1

# image shapes
image_shape = [225, 400, 3]
height, width, channels = image_shape


In [17]:
from CLIP import clip

# Available models for CLIP
# print( clip.available_models() ) # ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']

clipmodel, _ = clip.load('ViT-B/32')

100%|███████████████████████████████████████| 338M/338M [02:55<00:00, 2.02MiB/s]


AttributeError: 'VisionTransformer' object has no attribute 'image_resolution'

In [19]:
# input image shape?
img_shape = clipmodel.visual.input_resolution
print( img_shape ) # 224 -> n patches of (3, 224, 224) -> After CLIP -> img encoding -> n x 512

# set to eval as already trained
clipmodel.eval()

device = torch.device('cuda:0')
# emprt cache
torch.cuda.empty_cache()

224
